In [81]:
"""
수동으로 training과 동기화해야 합니다.
- training 시작 후, model 저장 폴더(model_output_path)가 생성되면 입력 후 코드 실행.
"""
from IPython.display import display
from utils import increment_path
import time
from glob import glob


def str_formatter(string, pp, cycle):
    pp.text(string)
plain = get_ipython().display_formatter.formatters['text/plain']
plain.for_type(str, str_formatter)

def display_dots(num_dots):
    str_dots = "." * num_dots
    dot_display.update(str_dots)
num_dots = 0
dot_display = display("", display_id=True)


# ********** CHANGE HERE (1/1): Path of pretrained reader model **********
model_output_path = "/opt/ml/outputs/models/no8_xlm_roberta_fine_tuning_continue"
# ************************************************************************
tokenizer_name = "xlm-roberta-large" # use huggingface modal name, if "Can't load tokenizer"
print("model from:", model_output_path)
eof_dir_path = model_output_path + "/zeof"

dataset_path = "/opt/ml/input/data/test_dataset"
prev_model_path = None
while True:
    num_dots += 1
    num_dots %= 10
    display_dots(num_dots)
    time.sleep(1)
    
    model_paths = sorted(glob(model_output_path + "/*"))
    if not model_paths:
        continue
    model_path = model_paths[-1]
    if model_path == eof_dir_path:
        print("Found EOF")
        break
    if model_path == prev_model_path:
        continue
    
    num_dots = 0
    display_dots(num_dots)
    print(model_path)
    prev_model_path = model_path
    
    result_name = "_".join(model_path.split("/")[-2:])
    test_output_dir = increment_path("/opt/ml/outputs/preds", "/no", result_name)
    !python inference.py\
        --tokenizer_name $tokenizer_name\
        --output_dir $test_output_dir\
        --dataset_path $dataset_path\
        --model_path $model_path\
        --topk 20
    
print("Inference End.")

....

model from: /opt/ml/outputs/models/no8_xlm_roberta_fine_tuning_continue
/opt/ml/outputs/models/no8_xlm_roberta_fine_tuning_continue/checkpoint-588
data is from /opt/ml/input/data/test_dataset
model is from /opt/ml/outputs/models/no8_xlm_roberta_fine_tuning_continue/checkpoint-588
model uses device: cuda:0
Dataset({
    features: ['id', 'question'],
    num_rows: 600
})
Lengths of unique contexts : 56607
Passage Embedding Loaded.
[Relevant documents exhaustive search.] done in 7.328 s
Making DataFrame dataset:: 600it [00:00, 14978.14it/s]
100%|███████████████████████████████████████████| 12/12 [00:57<00:00,  4.81s/ba]
Init Trainer...
100%|████████████████████████████████████| 12000/12000 [01:10<00:00, 170.31it/s]
INFO:utils_qa:Saving predictions to /opt/ml/outputs/preds/no17_no8_xlm_roberta_fine_tuning_continue_checkpoint-588/predictions.json.
INFO:utils_qa:Saving nbest_preds to /opt/ml/outputs/preds/no17_no8_xlm_roberta_fine_tuning_continue_checkpoint-588/nbest_predictions.json.
Job do

KeyboardInterrupt: 